In [ ]:
!pip install streamlit groq pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.6 MB/s eta 0:00:00


In [ ]:
# This command links your Colab notebook to your ngrok account.
# IMPORTANT: Replace "YOUR_NGROK_AUTHTOKEN_HERE" below with your actual ngrok authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
# Make sure to keep the quotes around your authtoken.

AUTHTOKEN = "35V9ZnsqiM6SHcI5t6aZHQirqq1_5JXWfBy3HotZm1CnVwJRu" # <--- PASTE YOUR NGROK AUTHTOKEN HERE
!ngrok authtoken "{AUTHTOKEN}"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
from groq import Groq

# --------------------------------------------------------
# Streamlit Page Config
# --------------------------------------------------------
st.set_page_config(page_title="LinkedIn Post Generator", layout="centered")

st.title("LinkedIn Post Generator: jiit 🚀")

# --------------------------------------------------------
# Sidebar for Groq API Key
# --------------------------------------------------------
with st.sidebar:
    groq_api_key = st.text_input("Enter Groq API Key:", type="password")
    st.markdown(
        "<a href='https://console.groq.com/keys' target='_blank'>Get your Groq API Key</a>",
        unsafe_allow_html=True
    )
    st.markdown("---")
    st.markdown("✨ **Extra Settings**")

    tone = st.selectbox("Post Tone",
                        ["Professional", "Motivational", "Storytelling", "Humorous", "Inspirational"])

    emoji_level = st.selectbox("Emoji Style",
                               ["None", "Light (1–2)", "Medium (3–5)", "Heavy (5+)"])

    hashtag_toggle = st.checkbox("Include Hashtags", value=True)

# --------------------------------------------------------
# Main Input Fields
# --------------------------------------------------------
col1, col2, col3 = st.columns(3)

with col1:
    topic = st.selectbox(
        "Topic",
        ("Job Search", "Tech Interview", "New Project", "Networking", "Industry News", "Personal Milestone")
    )

with col2:
    length = st.selectbox("Length", ("Short", "Medium", "Long"))

with col3:
    language = st.selectbox("Language", ("English", "Spanish", "French", "German", "Hindi"))

# --------------------------------------------------------
# Generate Button
# --------------------------------------------------------
submitted = st.button("Generate Post")

# --------------------------------------------------------
# Generate Post Logic
# --------------------------------------------------------
if submitted:
    if not groq_api_key:
        st.warning("⚠️ Please enter your Groq API key in the sidebar.")
    else:
        try:
            client = Groq(api_key=groq_api_key)

            system_prompt = f"""
            You are an expert LinkedIn content creator.

            Generate a LinkedIn post with the following settings:
            - Topic: {topic}
            - Tone: {tone}
            - Emoji Level: {emoji_level}
            - Length: {length}
            - Language: {language}
            - Include Hashtags: {hashtag_toggle}

            RULES:
            - Tone MUST strictly match: {tone}.
            - Emoji level:
                None = no emojis
                Light = 1–2 emojis
                Medium = 3–5 emojis
                Heavy = 5+ emojis
            - Length:
                Short = 1–2 sentences
                Medium = 3–5 sentences
                Long = 6+ sentences
            - If hashtags are enabled, add 5–10 trending hashtags.
            - Do NOT add explanations.
            - Output ONLY the LinkedIn post.
            """

            user_prompt = "Create the LinkedIn post now."

            with st.spinner("✨ Writing your LinkedIn post…"):
                response = client.chat.completions.create(
                    model="llama-3.3-70b-versatile",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    temperature=0.7,
                    max_tokens=800
                )

                post = response.choices[0].message.content

                # ---------------- Preview Styled Box ----------------
                st.subheader("✨ Your LinkedIn Post:")
                st.markdown(
                    f"""
                    <div style='background:#f7f7f7;padding:18px;border-radius:10px;
                                border:1px solid #dcdcdc;font-size:16px;line-height:1.6;
                                color:#222;'>
                    {post}
                    </div>
                    """,
                    unsafe_allow_html=True
                )

                # Character count
                st.info(f"Character Count: {len(post)} / 3000 (LinkedIn limit)")

                # Copy block
                st.code(post)

        except Exception as e:
            st.error(f"❌ Error: {e}")


Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null&  # Run streamlit in background

from pyngrok import ngrok
import time

# Terminate any existing tunnels
ngrok.kill()

# Start an http tunnel on port 8501
public_url = ngrok.connect(8501)
print(f"🚀 Your app is live! Click here: {public_url}")

🚀 Your app is live! Click here: NgrokTunnel: "https://valerianaceous-witless-charlsie.ngrok-free.dev" -> "http://localhost:8501"
